# UKBiobank metadata extractor

In [11]:
import os
import json

from lib.LORISIntrument import LORISIntrument
from lib.LORISField import LORISField

from lib.UKBiobankSchemas import UKBiobankSchemas

'''
  This helps translate the UKBiobank metadata into LORIS terms
'''
class LORISHelper:
    def __init__(self, schema):
        self.__schema = schema
        
    def generate_LINST_files(self, path = './instruments'):
        for obj in self.__schema.categories_with_fields:
            category = obj.get('category')
            fields = obj.get('fields')
            instrument = LORISIntrument(category, fields).as_LINST()
            if instrument is not None:
                filename = instrument[0].split('{@}')[1].strip() + '.linst'
                file = open(os.path.join('instruments', filename), 'w')
                for row in instrument:
                    file.write(row)
        
    def print_tests_batteries(self):
        insert_statement = 'INSERT INTO test_battery (Test_name, AgeMinDays, AgeMaxDays, Stage, SubprojectID, Visit_label) VALUES '
        values = []
        for obj in self.__schema.categories_with_fields:
            category = obj.get('category')
            fields = obj.get('fields')
            instrument = LORISIntrument(category, fields)
            table_name = instrument.table_name
            
            visits_set = set()
            for visits in instrument.instances:
                for v in visits:
                    visits_set.add(v)
            
            for visit_label in visits_set:
                values.append('({}, 0, 2147483647, "Visit", NULL, {})'.format(table_name, visit_label))
        
            if values:
                print(insert_statement)
                print(',\n'.join(values))
                print(';')
     
    def print_field_instances(self, field_id = ''):
        assert len(field_id), __class__.__name__ + ': field_id is required'
        try:
            raw_field = self.__schema.get_field(field_id)
        except KeyError:
            return 'KeyError: field not found'
        
        response = {
            "subproject_id": raw_field.get('instance_id'),
            "visit_labels": raw_field.get('instances')
        }
        print(str(response))
        
            
s = LORISHelper(UKBiobankSchemas(cachedir = os.path.join(os.getcwd(), '.cache')))
#s.generate_LINST_files()
#s.print_tests_batteries()            
s.print_field_instances('3')            

{'subproject_id': '2', 'visit_labels': ['init', 'rep1', 'img']}
